# Neuronexus Analysis v7

**Suite2P風 統合GUI**

1つのウィンドウで PLX読込 → LFPフィルタ → スパイクソーティング → 位相ロック解析 を順番に実行できます。

## 使い方
1. セル1: 環境設定 + 初期値の設定
2. セル2: GUI起動（ウィンドウを閉じるまでブロック）

初期値はGUI起動後にProjectStateに反映され、各ステップパネルに自動で読み込まれます。

In [ ]:
# === 環境設定 ===
import os

# PyTorch CUDA 用 DLL パス (必要な場合のみ)
torch_dll = r"c:\Users\wakel\anaconda3\envs\neoeeg\lib\site-packages\torch\lib"
if os.path.exists(torch_dll):
    os.add_dll_directory(torch_dll)

# NumPy 2.0+ 互換パッチ
import numpy_compat

# Matplotlib を tk バックエンドに設定
%matplotlib tk

print("Ready")

In [ ]:
# === 初期設定 (v6 の設定を引き継ぎ) ===
# ここを変更すればGUI起動時のデフォルト値になります

# --- PLXファイル・出力先 ---
PLX_FILE = r"\\Nagoya_AMCBNAS2\wakelab7\okita\whisker_analysis\20260114-1_Wildtype_S1BF\260114-4_Wildtype_S1BF_920_wisker\260114-4_Wildtype_S1BF_920_wisker.plx"
OUTPUT_DIR = r"\\Nagoya_AMCBNAS2\wakelab7\okita\whisker_analysis\20260114-1_Wildtype_S1BF\0209解析"

# --- LFP設定 ---
LFP_CONFIG = {
    "filter_enabled": True,
    "filter_type": "fir",
    "filter_lowcut": 0.1,
    "filter_highcut": 100.0,
    "filter_order": 4,
    "filter_fir_numtaps": 0,         # 0 = 自動
    "notch_enabled": True,
    "notch_freq": 60.0,
    "notch_Q": 60.0,
    "harmonic_removal_enabled": True,
    "harmonic_fundamental": 10.0,
    "harmonic_count": 4,
    "harmonic_q": 200,
    "noise_removal_enabled": False,
    "noise_file": "",
    "noise_threshold_db": 10.0,
    "bad_channel_detection": True,
    "bad_channel_threshold": 3.0,
    "manual_bad_channels": "",
    "motion_analysis": True,
    "motion_roi": "",
    "motion_percentile": 75.0,
    "motion_expand_sec": 0.1,
    "ica_enabled": True,
    "ica_noise_ratio_threshold": 1.5,
    "ica_max_remove": 4,
    "n_sessions": 9,
    "n_stim_per_session": 10,
    "baseline_pre_sec": 3.0,
    "post_duration_sec": 3.0,
    "power_freq_min": 0.5,
    "power_freq_max": 100.0,
    "fft_freq_max": 300.0,
    "show_plots": False,
    "save_plots": False,
    "bands": {
        "delta": [1.0, 4.0],
        "theta": [4.0, 8.0],
        "alpha": [8.0, 14.0],
        "beta": [14.0, 30.0],
        "gamma": [30.0, 80.0],
    },
}

# --- スパイクソーティング設定 ---
SPIKE_CONFIG = {
    "filter_low": "300.0",
    "filter_high": "3000.0",
    "filter_order": "4",
    "threshold_std": "4.0",
    "min_spike_interval_ms": "1.0",
    "artifact_threshold_std": "10.0",
    "pre_spike_ms": "0.5",
    "post_spike_ms": "1.0",
    "n_pca_components": "3",
    "max_clusters": "5",
    "min_cluster_size": "20",
    "isi_violation_threshold_ms": "2.0",
    "mua_isi_threshold": "5.0",
    "mua_snr_threshold": "2.0",
    "channel_spacing_um": "25.0",
    "n_templates": "4",
    "_method": "GMM (内蔵)",     # or "KiloSort4"
}

# --- 位相ロック解析設定 ---
PHASE_CONFIG = {
    "spike_source": "sorted",        # "plx" or "sorted"
    "spike_quality_filter": "sorted_only",  # "sorted_only", "all", "mua"
    "min_spikes": "50",
    "stim_artifact_window": "0.005",
    "channel_spacing_um": "25.0",
    "condition_analysis": True,
    "save_plots": True,
    "save_csv": True,
}

print(f"PLX: {os.path.basename(PLX_FILE)}")
print(f"Output: {OUTPUT_DIR}")
print(f"LFP bands: {list(LFP_CONFIG['bands'].keys())}")
print(f"Spike method: {SPIKE_CONFIG['_method']}")

In [ ]:
# === v7 統合GUI 起動 ===
# ノートブックの初期設定をProjectStateに注入してからGUI起動
from main_gui import MainGUI

app = MainGUI()

# --- 初期値をProjectStateに反映 ---
app.state.plx_file = PLX_FILE
app.state.output_dir = OUTPUT_DIR
app.state.lfp_config = LFP_CONFIG
app.state.spike_config = SPIKE_CONFIG
app.state.phase_config = PHASE_CONFIG

# サイドバーのプロジェクト情報を更新
app._update_sidebar()

# Step 0 パネルにパス反映
if "load" in app._panels:
    app._panels["load"].on_show()

print("GUI起動中... ウィンドウを閉じるとこのセルが完了します")
app.run()